CIFAR100-Classification

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow 

import os
import pickle
import numpy as np
import h5py

In [2]:
#download datasets
num_classes = 100
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar100.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
model = Sequential()

model.add(Conv2D(128, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = tensorflow.optimizers.RMSprop(learning_rate=0.0002, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [5]:
epochs = 20
#epoch++
data_augmentation = True
num_predictions = 20
batch_size = 64
validations = []

for i in range(epochs):
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=1)#,
                            #validation_data=(x_test, y_test))
        validations.append(model.evaluate(datagen.flow(x_test, y_test,
                                          batch_size=batch_size),
                                          steps=x_test.shape[0] // batch_size))

pickle.dump(validations, open("loss_validation.p",'wb'))
#跑过两组epoch（每组20个）

Using real-time data augmentation.
156/156 [==============================] - 39s 249ms/step - loss: 1.7663 - accuracy: 0.5349
Using real-time data augmentation.
156/156 [==============================] - 59s 376ms/step - loss: 1.7593 - accuracy: 0.5368
Using real-time data augmentation.
156/156 [==============================] - 62s 395ms/step - loss: 1.7109 - accuracy: 0.5509
Using real-time data augmentation.
156/156 [==============================] - 68s 439ms/step - loss: 1.6723 - accuracy: 0.5604
Using real-time data augmentation.
156/156 [==============================] - 36s 229ms/step - loss: 1.7143 - accuracy: 0.5485
Using real-time data augmentation.
156/156 [==============================] - 35s 225ms/step - loss: 1.6729 - accuracy: 0.5656
Using real-time data augmentation.
156/156 [==============================] - 34s 220ms/step - loss: 1.6713 - accuracy: 0.5568
Using real-time data augmentation.
156/156 [==============================] - 36s 229ms/step - loss: 1.6491 - a

In [9]:
epochs = 5
#epoch++
data_augmentation = True
num_predictions = 20
batch_size = 64
validations = []

for i in range(epochs):
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=1)#,
                            #validation_data=(x_test, y_test))
        validations.append(model.evaluate(datagen.flow(x_test, y_test,
                                          batch_size=batch_size),
                                          steps=x_test.shape[0] // batch_size))

pickle.dump(validations, open("loss_validation.p",'wb'))
#已跑过3组epoch（每组5个）

Using real-time data augmentation.
156/156 [==============================] - 33s 212ms/step - loss: 1.5741 - accuracy: 0.5986
Using real-time data augmentation.
156/156 [==============================] - 33s 212ms/step - loss: 1.5658 - accuracy: 0.5992
Using real-time data augmentation.
156/156 [==============================] - 32s 204ms/step - loss: 1.5821 - accuracy: 0.5930
Using real-time data augmentation.
156/156 [==============================] - 29s 184ms/step - loss: 1.5748 - accuracy: 0.5954
Using real-time data augmentation.
156/156 [==============================] - 28s 180ms/step - loss: 1.6259 - accuracy: 0.5845


In [11]:
epochs = 2
#epoch++
data_augmentation = True
num_predictions = 20
batch_size = 64
validations = []

for i in range(epochs):
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=1)#,
                            #validation_data=(x_test, y_test))
        validations.append(model.evaluate(datagen.flow(x_test, y_test,
                                          batch_size=batch_size),
                                          steps=x_test.shape[0] // batch_size))

pickle.dump(validations, open("loss_validation.p",'wb'))
#已跑过3组epoch（每组5个）

Using real-time data augmentation.
156/156 [==============================] - 28s 181ms/step - loss: 1.5852 - accuracy: 0.5971
Using real-time data augmentation.
156/156 [==============================] - 28s 179ms/step - loss: 1.5633 - accuracy: 0.5970


In [10]:
# Load label names to use in prediction results3-+
label_list_path = 'datasets/cifar-100-python/meta'

keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

print('Model Accuracy = %.4f' % (evaluation[1]))

predict_gen = model.predict(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['fine_label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['fine_label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

156/156 [==============================] - 27s 175ms/step - loss: 1.6421 - accuracy: 0.5773
Model Accuracy = 0.5773
Actual Label = mountain vs. Predicted Label = apple
Actual Label = forest vs. Predicted Label = crocodile
Actual Label = seal vs. Predicted Label = cup
Actual Label = mushroom vs. Predicted Label = turtle
Actual Label = sea vs. Predicted Label = caterpillar
Actual Label = tulip vs. Predicted Label = dinosaur
Actual Label = camel vs. Predicted Label = motorcycle
Actual Label = butterfly vs. Predicted Label = chimpanzee
Actual Label = cloud vs. Predicted Label = cloud
Actual Label = apple vs. Predicted Label = poppy
Actual Label = sea vs. Predicted Label = hamster
Actual Label = skunk vs. Predicted Label = willow_tree
Actual Label = streetcar vs. Predicted Label = crab
Actual Label = rocket vs. Predicted Label = otter
Actual Label = lamp vs. Predicted Label = road
Actual Label = lion vs. Predicted Label = couch
Actual Label = tulip vs. Predicted Label = flatfish
Actual Labe